# Dependencies
Execute the following line in the terminal before running this notebook:
`pip install -r requirements.txt`

# Exercise 1: Body temperature

In [17]:
# Code
def temp_tester(normalTemp): # Defines normal body temperature
    def tester(givenTemp): # Tests given temperature against normal body temperature
        if (givenTemp - 1 <= normalTemp <= givenTemp + 1): # if givenTemp is within 1 degree of normalTemp
            return True
        else:
            return False
    return tester

In [18]:
# Tests
human_tester = temp_tester(37) # Define 37 as normal temperature for humans
chicken_tester = temp_tester(41.1)

chicken_tester(42) # True
human_tester(42) # False
chicken_tester(43) # False
human_tester(35) # False
human_tester(98.6) # False

False

# Exercise 2: FASTQ

In [ ]:
# Code
def better_quality(char_a, char_b):
    char_a = chr(char_a) # Convert arguments to Unicode characters
    char_b = chr(char_b)
    pass 

def quality_measurement()

In [ ]:
# Tests

# Exercise 3: Intravenous fluids

# Exercise 4: Historical COVID-19 data
Data from [The New York Times](https://github.com/nytimes/covid-19-data), based on reports from state and local health agencies.

In [1]:
# Load historical data for COVID-19 cases by state
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv")

In [ ]:
# Plot new cases by state
import seaborn as sb
data_wide = data.pivot(index = "date", columns = "state", values = "cases").fillna(0)
sb.lineplot(data = data_wide)

In [ ]:
# Tests

# Exercise 5: MeSH data